##SETUP

In [2]:
import time
import pandas as pd
# Android ADB
from subprocess import check_output,Popen
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'started_scrcpy' not in vars():
    started_scrcpy=True
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process (Scrcpy window can be closed)
# internal
import bot_core
import bot_perception

import importlib
[importlib.reload(lib) for lib in [bot_core,bot_perception] ]


# Selected units in deck (order does not matter)
sel_units= ['crystal.png','chemist.png','monkey.png','dryad.png','hunter.png']
bot_core.select_units(sel_units,show=False)

bot = bot_core.Bot()

## RUN BOT

In [4]:
wait=0
combat = 0
grid_df =None
watch_ad = True
# Main loop
while(1):
    output = bot.battle_screen(True,True,floor=5)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                bot.mana_level([5,3,1],hero_power=True)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,merge_series,df_groups])
    else:
        if watch_ad:
            for i in range(5):
                bot.watch_ads()
            watch_ad = False
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

['23, 4/8 fighting, need more units!',
 'Average age: 43.93',
     grid_id         unit  probability  rank  rank_error position   Age
 0    icon_0  chemist.png        653.0     1      6286.0   [0, 0]  46.0
 1    icon_1    empty.png        999.0     0         0.0   [0, 1]  50.0
 2    icon_2    empty.png        700.0     0         0.0   [0, 2]  41.0
 3    icon_3    empty.png        700.0     0         0.0   [0, 3]  41.0
 4    icon_4    empty.png        700.0     0         0.0   [0, 4]  41.0
 5    icon_5    empty.png        700.0     0         0.0   [1, 0]  41.0
 6    icon_6    empty.png        999.0     0         0.0   [1, 1]  61.0
 7    icon_7    empty.png        700.0     0         0.0   [1, 2]  41.0
 8    icon_8  chemist.png        677.0     4       477.0   [1, 3]  46.0
 9    icon_9    empty.png        999.0     0         0.0   [1, 4]  46.0
 10  icon_10    empty.png        700.0     0         0.0   [2, 0]  41.0
 11  icon_11    empty.png        700.0     0         0.0   [2, 1]  41.0
 1

KeyboardInterrupt: 

# Play Game

In [2]:
# Restart game
bot.restart_RR()

## Quick restart

In [2]:
# Drop co-op game
bot.restart_RR(quick_disconnect=True)